In [1]:
!pip install pandas

     |████████████████████████████████| 10.1MB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 512kB 33.7MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 28.8MB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium --yes
import folium

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.0.1               |           py37_0         727 KB  conda-forge
    asn1crypto-0.24.0          |        py37_1003         154 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    cffi-1.12.3                |   py37h8022711_0         218 KB  conda-forge
    chardet-3.0.4              |        py37_1003         167 KB  conda-forge
    cryptography-2.6.1         |   py37h72c5cf5_0         607 KB  conda-forge
    folium-0.8.3               |             py_0          54 KB  conda-forge
    idna-2.8                   |        py37_1000         100 KB  conda-forge
    libblas-3.8.0              |       9_openblas           6 KB  conda-forge
    libcblas-3.8.

In [6]:
#Download store locations and remove unwanted data
df = pd.read_csv("https://raw.githubusercontent.com/ninjananjo/Clothing-Store-Competition/master/StoreLocations.csv", encoding = 'ISO-8859-1')
print(df.shape)
df = df[['TOWN','NAME','ID','COUNTRY','Address1','Postcode','Host','PNO','NSF']]
#df = df[df['ID'].str.startswith("TS")]
df.head(10)

(387, 56)


,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host,PNO,NSF
0,Z USA NEW YORK,NEW YORK (BROADWAY),TM7000,US,478 Broadway,10013,TS6000,3000,6900
1,SUTTON,SUTTON,TM7003,ENG,137-145 High Street,SM1 1JQ,TS0035,6023,1490
2,WORCESTER,WORCESTER 2,TM7006,ENG,Unit 3,WR1 2QE,TS0120,1811,1500
3,LONDON (KNIGHTSBRIDGE),KNIGHTSBRIDGE,TM7014,ENG,70 Brompton Road,SW3 1ER,TS1054,910,4106
4,PRESTON,PRESTON FISHERGATE,TM7017,ENG,125-127 Fishergate,PR1 2DA,TS0168,2552,3000
5,WORTHING,WORTHING,TM7021,ENG,21/23 Montague St,BN11 3BE,TS0028,1502,1306
6,SHREWSBURY,SHREWSBURY,TM7028,ENG,Unit Lru,SY1 1BN,TS0200,1824,1356
7,LUTON,LUTON,TM7036,ENG,65-67 Arndale Centre,LU1 3TP,TS0054,1474,2011
8,SCARBOROUGH,SCARBOROUGH,TM7067,ENG,Unit 29/30,YO11 1UE,TS0105,1525,1270
9,CHICHESTER,CHICHESTER,TM7069,ENG,89-91 East Street,PO19 1HA,TS1055,936,2458


In [22]:
#Group by Host
grouped_id = df.groupby('Host').apply(lambda x: ','.join(x.ID)).reset_index()
grouped_id.rename({0:'Branches'}, axis='columns', inplace=True)
df = df.groupby(['Host']).last()
df = df.merge(grouped_id, left_on='Host',right_on='Host')
print(df.shape)
df.head()

(200, 13)


,Host,TOWN,NAME,ID,COUNTRY,Address1,Postcode,PNO,NSF,Branches_x,Latitude,Longitude,Branches_y
0,TM7157,OLDHAM,OLDHAM,TM7157,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,53.541672,-2.112246,TM7157
1,TM7345,CHESTER,CHESTER,TS0324,ENG,12/14 St Michael's Square,CH1 1EE,1748,3300,"TM7345,TS0324",53.189646,-2.889252,TS0324
2,TM7439,Z EIRE DUBLIN,DUBLIN (GRAFTON ST),TM7439,EIR,41 Grafton Street,NaN,5028,3300,TM7439,48.056514,-4.111926,TM7439
3,TM7782,SHOREDITCH,SHOREDITCH,TM7782,ENG,98 Commercial Street,E1 6LZ,968,1150,TM7782,51.520023,-0.074251,TM7782
4,TM7815,DARLINGTON,DARLINGTON,TM7815,ENG,28-34 Northgate,DL1 1NR,1660,2478,TM7815,54.526350,-1.554351,TM7815


In [12]:
#get latitude and longitude data
def getLocation(storeName, postcode, country):
    
    if country == 'ENG': country = "UK"
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(str(postcode)+" "+country)
    location_alt = geolocator.geocode(storeName+" "+country)
    try:
        lat = location.latitude
        lng = location.longitude
    except:
        try:
            lat = location_alt.latitude
            lng = location_alt.longitude
        except:
            lat = 0
            lng = 0
            
    return(lat,lng)

In [13]:
latitude = []
longitude = []

for storeName, postcode, country in zip(df['NAME'],df['Postcode'],df['COUNTRY']):
    lati, lngi = getLocation(storeName, postcode, country)
    latitude.append(lati)
    longitude.append(lngi)

df.loc[:,'Latitude']= latitude
df.loc[:,'Longitude']= longitude
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df.head()

200


,Host,TOWN,NAME,ID,COUNTRY,Address1,Postcode,PNO,NSF,Branches,Latitude,Longitude
0,TM7157,OLDHAM,OLDHAM,TM7157,ENG,Unit 3,OL1 1XE,1361,2200,TM7157,53.541672,-2.112246
1,TM7345,CHESTER,CHESTER,TS0324,ENG,12/14 St Michael's Square,CH1 1EE,1748,3300,"TM7345,TS0324",53.189646,-2.889252
2,TM7439,Z EIRE DUBLIN,DUBLIN (GRAFTON ST),TM7439,EIR,41 Grafton Street,NaN,5028,3300,TM7439,48.056514,-4.111926
3,TM7782,SHOREDITCH,SHOREDITCH,TM7782,ENG,98 Commercial Street,E1 6LZ,968,1150,TM7782,51.520023,-0.074251
4,TM7815,DARLINGTON,DARLINGTON,TM7815,ENG,28-34 Northgate,DL1 1NR,1660,2478,TM7815,54.526350,-1.554351
5,TM7939,KINGSTON,KINGSTON UPON THAMES,TM7939,ENG,40 Clarence Street,KT1 1NR,7017,2097,TM7939,51.410628,-0.302573
6,TM8134,GLASGOW,GLASGOW BUCHANAN ST,TS2876,SCO,The George Unit 1& 2,G1 2NG,404,8404,"TM8134,TS2876",55.863863,-4.253183
7,TS0001,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,8512,69207,"TM7329,TS0001",51.515534,-0.141352
8,TS0003,BRENT CROSS,BRENT CROSS,TS0003,ENG,"Unit 72, Brent Cross Shopping Centre",NW4 3FP,780,11237,"TM7218,TS0003",51.574492,-0.221565
9,TS0004,LIVERPOOL,LIVERPOOL,TS0004,ENG,22/36 Church Street,L1 3AY,1479,20200,"TM7256,TS0004",53.404741,-2.982541


In [14]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version
LIMIT = 400 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
categoryId = '4bf58dd8d48988d1f2941735'

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    
    store_list = []
    lat_list = []
    lng_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name 
        for i in range(0,resultsVen.shape[0]):
            store_list.append(name)
            lat_list.append(lat)
            lng_list.append(lng)

        store = pd.DataFrame([store_list,lat_list,lng_list]).T
        nearbyVenues = pd.concat([store, resultsVenCat], axis=1)

    return(nearbyVenues)

In [16]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for storeId in df['Host'].unique():
    store_info = df[df['Host']==storeId]
    regions_venues = regions_venues.append(getNearbyVenues(names=store_info['Host'],
                                                  latitudes=store_info['Latitude'],
                                                  longitudes=store_info['Longitude'],
                                                  radius=radius),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={0:'Host ID',
                          1:'Host Lat',
                          2:'Host Lng',
                          'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

print(regions_venues.shape[0])
regions_venues.head()

5468


,Host ID,Host Lat,Host Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,TM7157,53.5417,-2.11225,4d0f4ef7cf09a143e980240f,Sports Direct,76,"[15-17 High St., Oldham, Lancashire, OL1 3AJ, ...",53.542042,-2.113216,Sporting Goods Shop
1,TM7157,53.5417,-2.11225,5af008d0628c83002cdb1011,DW Fitness First,12659,"[Gelderd Road, Birstall, Leeds, WF17 9TD, Unit...",53.470197,-2.260968,Gym
2,TM7157,53.5417,-2.11225,581c781c77259217e0937626,Kids Foot Locker,10824,"[The Arndale Centre, unit 10B, Cromford Court ...",53.483238,-2.242959,Shoe Store
3,TM7157,53.5417,-2.11225,5bfbd8b4ea1e44002ce12dd3,Trespass,10761,"[Manchester Arndale, MANCHESTER, M4 2HU, Unite...",53.482360,-2.240630,Outdoor Supply Store
4,TM7157,53.5417,-2.11225,5a8ecd550881587526bae41b,JD Sports,9173,"[Unit 18 Crown Point North Denton, Denton, Tam...",53.459318,-2.117344,Sporting Goods Shop


In [18]:
jdsports = regions_venues[regions_venues['Venue Name']=='JD Sports']
jdsports.to_csv('jdsports_proximty.csv')
print(jdsports.shape[0])
jdsports.head()

492


,Host ID,Host Lat,Host Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
4,TM7157,53.5417,-2.11225,5a8ecd550881587526bae41b,JD Sports,9173,"[Unit 18 Crown Point North Denton, Denton, Tam...",53.459318,-2.117344,Sporting Goods Shop
5,TM7157,53.5417,-2.11225,4b7ec528f964a52010fe2fe3,JD Sports,10794,"[16-18 Arndale Centre, Barbarolli Mall, Manche...",53.483574,-2.242812,Sporting Goods Shop
8,TM7157,53.5417,-2.11225,4c6aca2f897b1b8d72b1b217,JD Sports,2522,"[Unit 3 Centre Retail Park Elk Mill, Oldham, L...",53.558100,-2.138509,Sporting Goods Shop
16,TM7157,53.5417,-2.11225,4da1ab35b3e7236a011e0e79,JD Sports,10027,"[Unit B Central Retail Park Gt Ancoats St, Man...",53.482402,-2.226314,Sporting Goods Shop
24,TM7157,53.5417,-2.11225,501620dfe4b0ad6ab37d6045,JD Sports,9006,"[38 Yorkshire Street, Rochdale, OL16 1JN, Unit...",53.617893,-2.157928,Sporting Goods Shop


In [ ]:
df_map = jdsports
closest_jd = df_map.groupby(['Host ID'], as_index =False).min()
closest_jd = ['Host ID','Host Lat','Host Lng','Venue Distance','Venue Address','Venue Lat','Venue Lng']
closest_jd.head()

In [20]:
m = folium.Map(location=[53.4626, -2.34612], zoom_start=6)

for branch, branch_lat, branch_lng, proximity in zip(closest_jd['Host ID'], closest_jd['Host Lat'], closest_jd['Host Lng'], closest_jd['Venue Distance']):
    if proximity < 2000:
        colour = 'green'
    else: 
        colour = 'red'
    folium.Marker(location=[branch_lat, branch_lng], popup=branch).add_to(m)
    folium.Circle(location=[branch_lat, branch_lng], radius=2000, fill=False, color=colour).add_to(m)

for branch_lat, branch_lng, jd_lat, jd_lng in zip(df_map['Host Lat'], df_map['Host Lng'], df_map['Venue Lat'], df_map['Venue Lng']):
    folium.Circle(location=[jd_lat, jd_lng], popup='JD Sports', radius=50, fill=True, color='black').add_to(m)

m

In [21]:
#Stores without a JD Sports within a 2km radius
closest_jd[closest_jd['Venue Distance']>2000]

,Host ID,Host Lat,Host Lng,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
0,TM7157,53.541672,-2.112246,4b7ec528f964a52010fe2fe3,JD Sports,2522,"[16-18 Arndale Centre, Barbarolli Mall, Manche...",53.459318,-2.242812,Sporting Goods Shop
2,TM7782,51.520023,-0.074251,4ac518f1f964a520ecad20e3,JD Sports,4555,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
7,TS0003,51.574492,-0.221565,4ac518f1f964a520ecad20e3,JD Sports,4334,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.280536,Sporting Goods Shop
11,TS0014,51.461030,-0.304219,4c0f573a2466a5931e797a21,JD Sports,5614,"[20 Church Street, Kingston, Greater London, K...",51.410596,-0.304978,Sporting Goods Shop
23,TS0062,51.751444,-0.337992,4c39a9973849c928aaa1c1b1,JD Sports,6825,"[Hatfield Park Plaza, Comet Way (Comet Way), H...",51.749470,-0.471251,Sporting Goods Shop
24,TS0063,52.291022,-1.534741,4d9dd22f5c33a35d530b0aa1,JD Sports,3446,[United Kingdom],52.320816,-1.548500,Sporting Goods Shop
28,TS0080,49.206646,-2.198538,4c7f9c0cd51ea143b1005b9f,JD Sports,7073,"[42 King St, St Helier, JE2 4WE, Jersey]",49.184662,-2.107299,Sporting Goods Shop
35,TS0122,53.760275,-1.570111,4bf90c63b182c9b6da5b785a,JD Sports,4482,"[132 Briggate, Leeds, LS1 6HF, United Kingdom]",53.683906,-1.753280,Sporting Goods Shop
57,TS0185,-33.648333,150.734444,5b7f3ce50457b7002caf4c21,JD Sports,12103,"[316 Level 2 Westfield Penrith, 585 High Stree...",-33.751662,150.693781,Sporting Goods Shop
62,TS0309,51.466615,-0.096800,4ac518f1f964a520ecad20e3,JD Sports,6171,"[197-203 Oxford Street, West End, Greater Lond...",51.428388,-0.167446,Sporting Goods Shop
